In [99]:
# Import modules and clean data
import pandas as pd
import numpy as np

cleveland = pd.read_csv('processed.cleveland.data')
switzerland = pd.read_csv('processed.switzerland.data')
df = pd.concat([cleveland, switzerland])
df.replace('?', np.NAN, inplace=True)
df.dropna(thresh=13, inplace = True)
df.reset_index(inplace=True)
df.drop('index', axis = 1, inplace = True)
df[:] = df[:].astype(float)
df.ca.fillna( df.ca.median(), inplace = True)
df.thal.fillna(df.thal.mean(), inplace = True)
df.ca = df.ca.astype(int)
df.num = df.num.astype(int)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,68.0,1.0,4.0,135.0,0.0,0.0,1.0,120.0,1.0,0.0,1.0,0,7.0,3
342,69.0,1.0,4.0,135.0,0.0,0.0,0.0,130.0,0.0,0.0,2.0,0,6.0,1
343,70.0,1.0,4.0,115.0,0.0,0.0,1.0,92.0,1.0,0.0,2.0,0,7.0,1
344,70.0,1.0,4.0,140.0,0.0,1.0,0.0,157.0,1.0,2.0,2.0,0,7.0,3


In [100]:
# Create copy datafram with binary clasification
df_ = df.copy()
df_.num.replace([1,2,3,4], 1, inplace= True)
print(df_.num.value_counts(),'\n', df.num.value_counts())

1    181
0    165
Name: num, dtype: int64 
 0    165
1     67
3     50
2     48
4     16
Name: num, dtype: int64


In [101]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('num',axis=1), df['num'], test_size=0.35, random_state=1234)
X_train_, X_test_, y_train_, y_test_ = train_test_split(df_.drop('num',axis=1), df_['num'], test_size=0.35, random_state=1234)

In [102]:
# Fit random forest models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

RFC_advanced = RandomForestClassifier(
    n_estimators=270, min_samples_split=3,  criterion="entropy", random_state=1234
    ).fit(X_train, y_train)

RFC_basic = RandomForestClassifier(
    n_estimators=30, min_samples_split=3,  criterion="entropy", random_state=1234
    ).fit(X_train_, y_train_)


# Score the models
RFC_advanced_scores = {
    "Accuracy":accuracy_score(y_test, RFC_advanced.predict(X_test)),
    "Precision":precision_score(y_test, RFC_advanced.predict(X_test), average = "weighted"),
    "Recall":recall_score(y_test, RFC_advanced.predict(X_test), average = "weighted")
}
RFC_basic_scores = {
    "Accuracy":accuracy_score(y_test_, RFC_basic.predict(X_test_)),
    "Precision":precision_score(y_test_, RFC_basic.predict(X_test_)),
    "Recall":recall_score(y_test_, RFC_basic.predict(X_test_))
}
print(f'Scores for dataset with 4 classifications: \n{RFC_advanced_scores}')
print(f'Scores for dataset with 2 classifications: \n{RFC_basic_scores}')

Scores for dataset with 4 classifications: 
{'Accuracy': 0.4918032786885246, 'Precision': 0.3864900468384075, 'Recall': 0.4918032786885246}
Scores for dataset with 2 classifications: 
{'Accuracy': 0.8524590163934426, 'Precision': 0.9152542372881356, 'Recall': 0.8059701492537313}
